<a href="https://colab.research.google.com/github/ftnext/practice-dl-nlp/blob/master/llmjp/fine_tuning/hf_blog_gemma_peft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/blog/gemma-peft

## Setup

In [23]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

In [24]:
!pip install -q transformers[torch] datasets accelerate trl peft bitsandbytes

## Load model

In [25]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [26]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [27]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"": 0})

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2b/snapshots/68e273d91b1d6ea57c9e6024c4f887832f7b43fa/config.json
Model config GemmaConfig {
  "_name_or_path": "google/gemma-2b",
  "architectures": [
    "GemmaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "eos_token_id": 1,
  "head_dim": 256,
  "hidden_act": "gelu",
  "hidden_activation": null,
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 16384,
  "max_position_embeddings": 8192,
  "model_type": "gemma",
  "num_attention_heads": 8,
  "num_hidden_layers": 18,
  "num_key_value_heads": 1,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.44.2",
  "use_cache": true,
  "vocab_size": 256000
}

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing GemmaForCausalLM.

All the weights of GemmaForCausalLM were initialized from the model checkpoint at google/gemma-2b.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GemmaForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2b/snapshots/68e273d91b1d6ea57c9e6024c4f887832f7b43fa/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 2,
  "eos_token_id": 1,
  "pad_token_id": 0
}



In [28]:
model.config._attn_implementation

'sdpa'

## Before SFT

In [29]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

loading file tokenizer.model from cache at /root/.cache/huggingface/hub/models--google--gemma-2b/snapshots/68e273d91b1d6ea57c9e6024c4f887832f7b43fa/tokenizer.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2b/snapshots/68e273d91b1d6ea57c9e6024c4f887832f7b43fa/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2b/snapshots/68e273d91b1d6ea57c9e6024c4f887832f7b43fa/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2b/snapshots/68e273d91b1d6ea57c9e6024c4f887832f7b43fa/tokenizer_config.json


In [30]:
tokenizer.pad_token = tokenizer.eos_token

In [31]:
inputs = tokenizer("Quote: Imagination is more", return_tensors="pt").to("cuda:0")
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Imagination is more important than knowledge. Knowledge is limited. Imagination encircles the world.

- Albert Einstein

The


In [32]:
from transformers import DataCollatorForLanguageModeling

In [33]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

## Load dataset

In [34]:
from datasets import load_dataset

In [35]:
dataset = load_dataset("Abirate/english_quotes", split="train")

In [36]:
dataset["quote"][:2], dataset["author"][:2]

(['“Be yourself; everyone else is already taken.”',
  "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”"],
 ['Oscar Wilde', 'Marilyn Monroe'])

## Configuration

In [37]:
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

In [38]:
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

training_args = SFTConfig(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    max_steps=10,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    output_dir="outputs",
    optim="paged_adamw_8bit",
    log_level="info",
    hub_model_id="gemma-2b-peft-tutorial-english-quotes"
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [39]:
def formatting_prompts_func(dataset):
    return [
        f"Quote: {quote}\nAuthor: {author}<eos>"
        for quote, author in zip(dataset["quote"], dataset["author"])
    ]

## Train

In [40]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    peft_config=lora_config,
    formatting_func=formatting_prompts_func,  # 初期化時にdatasetの変換が行われた
    tokenizer=tokenizer,
    data_collator=data_collator,
)

PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:407: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend


In [41]:
trainer.train()

***** Running training *****
  Num examples = 2,508
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 4
  Total optimization steps = 10
  Number of trainable parameters = 9,805,824


Step,Training Loss
1,2.641400
2,1.980000
3,2.530700
4,2.628700
5,2.485500
6,2.545200
7,2.569600
8,2.145700
9,2.908000
10,2.393300


Saving model checkpoint to outputs/checkpoint-10
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2b/snapshots/68e273d91b1d6ea57c9e6024c4f887832f7b43fa/config.json
Model config GemmaConfig {
  "architectures": [
    "GemmaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "eos_token_id": 1,
  "head_dim": 256,
  "hidden_act": "gelu",
  "hidden_activation": null,
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 16384,
  "max_position_embeddings": 8192,
  "model_type": "gemma",
  "num_attention_heads": 8,
  "num_hidden_layers": 18,
  "num_key_value_heads": 1,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.44.2",
  "use_cache": true,
  "vocab_size": 256000
}

tokenizer config file saved in outputs/checkpoint-10/tokenizer_config.json
Special tokens file saved in 

TrainOutput(global_step=10, training_loss=2.4827977657318114, metrics={'train_runtime': 14.1803, 'train_samples_per_second': 2.821, 'train_steps_per_second': 0.705, 'total_flos': 21127850065920.0, 'train_loss': 2.4827977657318114, 'epoch': 0.01594896331738437})

## After SFT

In [42]:
inputs = tokenizer("Quote: Imagination is more", return_tensors="pt").to("cuda:0")
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Imagination is more important than knowledge. Knowledge is limited. Imagination encircles the world.

Author: Albert Einstein




In [43]:
# 所定の書式から始まらないとファインチューンの効果は得られなさそう
inputs = tokenizer("Imagination is more", return_tensors="pt").to("cuda:0")
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Imagination is more important than knowledge.

Albert Einstein

The following is a list of the most important and most interesting


In [44]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.33.0
aiohappyeyeballs                 2.4.0
aiohttp                          3.10.5
aiosignal                        1.3.1
alabaster                        0.7.16
albucore                         0.0.14
albumentations                   1.4.14
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.18.0
asn1crypto                       1.5.1
astropy                          6.1.3
astropy-iers-data                0.2024.8.27.10.28.29
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            24.2.0
audioread             

## Save

In [45]:
# trainer.push_to_hub()